In [1]:
import os
import pickle
import argparse
import numpy as np
from model import Options, Seq2SeqAttn


In [2]:
# Parse the command line arguments.
parser = argparse.ArgumentParser()
parser.add_argument('--data_path', type = str, default = '../pre-data/',
                    help = 'the directory to the data')

parser.add_argument('--word_embeddings_path', type = str, default = '../pre-data/word_embeddings.npy',
                    help = 'the directory to the pre-trained word embeddings')
parser.add_argument('--VAD_path', type = str, default = '../pre-data/VAD.npy',
                    help = 'the directory to VAD')
parser.add_argument('--tf_path', type = str, default = '../pre-data/tf.npy',
                    help = 'the directory to term frequency')

parser.add_argument('--num_epochs', type = int, default = 1,
                    help = 'the number of epochs to train the data')
parser.add_argument('--batch_size', type = int, default = 1,
                    help = 'the batch size')
parser.add_argument('--learning_rate', type = float, default = 0.001,
                    help = 'the learning rate')
parser.add_argument('--beam_width', type = int, default = 256,
                    help = 'the beam width when decoding')
parser.add_argument('--word_embed_size', type = int, default = 256,
                    help = 'the size of word embeddings')
parser.add_argument('--n_hidden_units_enc', type = int, default = 256,
                    help = 'the number of hidden units of encoder')
parser.add_argument('--n_hidden_units_dec', type = int, default = 256,
                    help = 'the number of hidden units of decoder')
# ? attn_depth
parser.add_argument('--attn_depth', type = int, default = 128,
                    help = 'attention depth')
parser.add_argument('--restore_path', type = str, default = '../model_dailydialog_rf',
                    help = 'the path to restore the trained model')
parser.add_argument('--restore_epoch', type = int, default = 0,
                    help = 'the epoch to restore')

parser.add_argument('--save_path', type = str, default = '../model_dailydialog_rf',
                    help = 'the path to save the trained model to')

# args = parser.parse_args()
args, unknown = parser.parse_known_args()


In [3]:
def read_data(data_path):
    def load_np_files(path):
        my_set = {}
        my_set['enc_input'] = np.load(os.path.join(path, 'enc_input.npy'))
        my_set['dec_input'] = np.load(os.path.join(path, 'dec_input.npy'))
        my_set['target'] = np.load(os.path.join(path, 'target.npy'))
        my_set['enc_input_len'] = np.load(os.path.join(path, 'enc_input_len.npy'))
        my_set['dec_input_len'] = np.load(os.path.join(path, 'dec_input_len.npy'))
        return my_set
    train_set = load_np_files(os.path.join(data_path, 'train'))
    valid_set = load_np_files(os.path.join(data_path, 'validation'))
    with open(os.path.join(data_path, 'token2id.pickle'), 'rb') as file:
        token2id = pickle.load(file)

    return train_set, valid_set, token2id

In [4]:
if __name__ == '__main__':
    train_set, valid_set, token2id = read_data(args.data_path)
    max_uttr_len_enc = train_set['enc_input'].shape[1]
    max_uttr_len_dec = train_set['dec_input'].shape[1]

    word_embeddings = np.load(args.word_embeddings_path)
    VAD = np.load(args.VAD_path)
    termfreq = np.load(args.tf_path)
    termfreq = termfreq.reshape(-1,1)

    options = Options(mode = 'TRAIN',
                      num_epochs = args.num_epochs,
                      batch_size = args.batch_size,
                      learning_rate = args.learning_rate,
                      beam_width = args.beam_width,
                      vocab_size = len(token2id),
                      max_uttr_len_enc = max_uttr_len_enc,
                      max_uttr_len_dec = max_uttr_len_dec,
                      go_index = token2id['<go>'],
                      eos_index = token2id['<eos>'],
                      word_embed_size = args.word_embed_size,
                      n_hidden_units_enc = args.n_hidden_units_enc,
                      n_hidden_units_dec = args.n_hidden_units_dec,
                      attn_depth = args.attn_depth,
                      word_embeddings = word_embeddings,
                      corpus_size = VAD.shape[0])
    model = Seq2SeqAttn(options)

    for var in model.tvars:
        print(var.name)

    if args.restore_epoch > 0:
        model.restore(os.path.join(args.restore_path, 'model_epoch_{:03d}.ckpt'.format(args.restore_epoch)))
    else:
        model.init_tf_vars()
    model.train(train_set, VAD,termfreq, args.save_path, args.restore_epoch, valid_set)


Building the TensorFlow graph...
embedding/word_embeddings:0
encoding/rnn/gru_cell/gates/kernel:0
encoding/rnn/gru_cell/gates/bias:0
encoding/rnn/gru_cell/candidate/kernel:0
encoding/rnn/gru_cell/candidate/bias:0
decoding/memory_layer/kernel:0
decoding/attention_v:0
decoding/attention_Wb:0
decoding/my_bahdanau_attention/query_layer/kernel:0
decoding/attention_wrapper/gru_cell/gates/kernel:0
decoding/attention_wrapper/gru_cell/gates/bias:0
decoding/attention_wrapper/gru_cell/candidate/kernel:0
decoding/attention_wrapper/gru_cell/candidate/bias:0
decoding/dense/kernel:0
decoding/dense/bias:0
TensorFlow variables initialized.
Start to train the model...
Epoch 001/001, valid ppl = None, batch 0001/0508, train loss = 7.15625524520874
Epoch 001/001, valid ppl = None, batch 0002/0508, train loss = nan
Epoch 001/001, valid ppl = None, batch 0003/0508, train loss = nan
Epoch 001/001, valid ppl = None, batch 0004/0508, train loss = nan
Epoch 001/001, valid ppl = None, batch 0005/0508, train loss

Epoch 001/001, valid ppl = None, batch 0114/0508, train loss = nan
Epoch 001/001, valid ppl = None, batch 0115/0508, train loss = nan
Epoch 001/001, valid ppl = None, batch 0116/0508, train loss = nan
Epoch 001/001, valid ppl = None, batch 0117/0508, train loss = nan
Epoch 001/001, valid ppl = None, batch 0118/0508, train loss = nan
Epoch 001/001, valid ppl = None, batch 0119/0508, train loss = nan
Epoch 001/001, valid ppl = None, batch 0120/0508, train loss = nan
Epoch 001/001, valid ppl = None, batch 0121/0508, train loss = nan
Epoch 001/001, valid ppl = None, batch 0122/0508, train loss = nan
Epoch 001/001, valid ppl = None, batch 0123/0508, train loss = nan
Epoch 001/001, valid ppl = None, batch 0124/0508, train loss = nan
Epoch 001/001, valid ppl = None, batch 0125/0508, train loss = nan
Epoch 001/001, valid ppl = None, batch 0126/0508, train loss = nan
Epoch 001/001, valid ppl = None, batch 0127/0508, train loss = nan
Epoch 001/001, valid ppl = None, batch 0128/0508, train loss =

Epoch 001/001, valid ppl = None, batch 0237/0508, train loss = nan
Epoch 001/001, valid ppl = None, batch 0238/0508, train loss = nan
Epoch 001/001, valid ppl = None, batch 0239/0508, train loss = nan
Epoch 001/001, valid ppl = None, batch 0240/0508, train loss = nan
Epoch 001/001, valid ppl = None, batch 0241/0508, train loss = nan
Epoch 001/001, valid ppl = None, batch 0242/0508, train loss = nan
Epoch 001/001, valid ppl = None, batch 0243/0508, train loss = nan
Epoch 001/001, valid ppl = None, batch 0244/0508, train loss = nan
Epoch 001/001, valid ppl = None, batch 0245/0508, train loss = nan
Epoch 001/001, valid ppl = None, batch 0246/0508, train loss = nan
Epoch 001/001, valid ppl = None, batch 0247/0508, train loss = nan
Epoch 001/001, valid ppl = None, batch 0248/0508, train loss = nan
Epoch 001/001, valid ppl = None, batch 0249/0508, train loss = nan
Epoch 001/001, valid ppl = None, batch 0250/0508, train loss = nan
Epoch 001/001, valid ppl = None, batch 0251/0508, train loss =

Epoch 001/001, valid ppl = None, batch 0360/0508, train loss = nan
Epoch 001/001, valid ppl = None, batch 0361/0508, train loss = nan
Epoch 001/001, valid ppl = None, batch 0362/0508, train loss = nan
Epoch 001/001, valid ppl = None, batch 0363/0508, train loss = nan
Epoch 001/001, valid ppl = None, batch 0364/0508, train loss = nan
Epoch 001/001, valid ppl = None, batch 0365/0508, train loss = nan
Epoch 001/001, valid ppl = None, batch 0366/0508, train loss = nan
Epoch 001/001, valid ppl = None, batch 0367/0508, train loss = nan
Epoch 001/001, valid ppl = None, batch 0368/0508, train loss = nan
Epoch 001/001, valid ppl = None, batch 0369/0508, train loss = nan
Epoch 001/001, valid ppl = None, batch 0370/0508, train loss = nan
Epoch 001/001, valid ppl = None, batch 0371/0508, train loss = nan
Epoch 001/001, valid ppl = None, batch 0372/0508, train loss = nan
Epoch 001/001, valid ppl = None, batch 0373/0508, train loss = nan
Epoch 001/001, valid ppl = None, batch 0374/0508, train loss =

KeyboardInterrupt: 